In [162]:
import IPython, jinja2, bokeh, bokeh.plotting, json
from coffeetools import coffee
from bokeh.sampledata import iris

In [163]:
#nbviewer
IPython.display.Javascript(coffee.compile("""
window.require.config
    paths: 
        d3: '//d3js.org/d3.v3.min'
        jquery: '/static/components/jquery/dist/jquery.min'
        jqueryui: 'https://cdn.jsdelivr.net/jquery.ui/1.11.4/jquery-ui.min'
        baobab: 'https://cdn.rawgit.com/Yomguithereal/baobab/master/build/baobab.min'
    shim: 
        jqueryui: 
            exports: '$'
            deps: ['jquery'] 
"""))
bokeh.plotting.output_notebook(resources=bokeh.resources.CDN)

BokehJS successfully loaded.

# Create some data 

Export the DataFrame using the split orientations.  This provides the ``index``,``columns``, and ``data``.  We can append the column metadata easily

In [ ]:
df=iris.flowers
payload = df.to_dict(orient='split')
payload['index'] = [float(v) for v in payload['index']]

# Make a Bokeh plot

Make a Bokeh plot to use as Boiler plate in the Javascript.  Javascript only knows bokeh as a structured JSON serialization.  Embed the serialization and modify the Bokeh state using ``Bokeh.Collections(type).get(id)``.

In [ ]:
p = bokeh.plotting.figure(width=200,height=200)
p.add_tools(bokeh.models.TapTool())
source = bokeh.models.ColumnDataSource(df)
scatter = p.circle(
    source = source,
    x = df.columns[0],
    y = df.columns[1],
    size=10,    
)
plot = {
    'figure': p.ref,
    'renderer': scatter.ref,
    'source': source.ref,
    'x_range': p.x_range.ref,
    'y_range': p.y_range.ref,
}

> The classes below have manage shared data between different selections in the DOM.  This representation makes it easy to wire interactivity last.

In [ ]:
IPython.display.Javascript(coffee.compile(jinja2.Template("""    
require ['d3','baobab'], (d3,Baobab)->    
    class window.Manager
        ### 
        One Data Source for many selections
        Attaches event driven data tress to selections
        Selections in a manager share a data source
        ###
        constructor: (@data)->
            ### @data changes state many times ###
            @DataSource = new Manager.DataSource @data
            @Selections = []

        attach: (selection,SelectionType,name,config)->
            ### Attach a new selection to the manager ###
            selections_list = @Selections.filter (s)=> s.selection.node() == selection.node()
            if selections_list.length == 0
                id = @Selections.push new SelectionType selection, name, @DataSource.tree, config
                @Selections[id-1]
            else
                selections_list[0]
    class Manager.DataSource 
        ### 
        A data source manages raw and derived data for different selections.  A data source can 
        have many selections.  The raw data is expected to be in a split format

        data: mxn
        columns: n x 1
        index: m x 1
        ###

        constructor: (@data)->
            console.log Baobab
            @tree = new Baobab @data
            if @tree.select('columns').exists()
                for column,column_index in columns = @get 'columns' 
                    ### Create Dynamic Nodes for Each Column Data Source ###
                    @addColumnDataSource column
            if @tree.select('index').exists()    
                ### Create a data source for the index ###
                @addColumnDataSource 'index', Baobab.monkey ['index'], (data)-> data

        updateIndex: (new_index)->
            ### Set sorted data and indices ###
            @set 'data', d3.permute @get('data'), new_index.map (value)=> 
                    @ColumnDataSource('index').indexOf value
            @set 'index', new_index
            this

        addDerivedDataSource: (name, columns, f )->
            ### return the new data source ###
            cursor = @tree.select('columns')
            cursor.push name
            @addColumnDataSource name, Baobab.monkey columns.map((value)=>['ColumnDataSource',value,'data'])..., f
            data = cursor.get().map((column_name)=> @ColumnDataSource column_name)
            @tree.set 'data', d3.zip data...
            @ColumnDataSource name

        ColumnDataSource: (column_name)->
            if Array.isArray column_name
                d3.zip column_name.map((c)=>@get('ColumnDataSource', c,'data'))...
            else
                Array @get('ColumnDataSource', column_name,'data')...

        addColumnDataSource: (column, monkey)->
            monkey ?= Baobab.monkey ['data'], ['.','name'], ['columns'], (data,name,columns)-> 
                column_index = columns.indexOf name
                data.map (value)=> value[column_index]
            @set ['ColumnDataSource', column], 
                name: column
                data: monkey

        sort: (columns,direction='ascending')->
            ### Multisort on an  ###
            MultiSort = (a,b,direction='ascending',i=0)->
                ### Multisorting function in d3 ###
                [a[i],b[i]] = [parseFloat(a[i]),parseFloat(b[i])]
                if a[i] == b[i] then MultiSort a, b, direction, i+1
                d3[direction] a[i],b[i]

            columns = if not Array.isArray columns then [columns] else columns
            sorted = d3.zip columns.map((c)=> @ColumnDataSource c)..., @ColumnDataSource 'index'
                .sort (a,b)-> MultiSort a,b,direction
            @updateIndex sorted.map (value)-> value[columns.length]     
            this

        order: ()-> 
            ### Order the original indices ###
            @sort('index')

        shuffle: ()-> 
            ### Randomly Sort the Indices ###
            @updateIndex d3.shuffle @ColumnDataSource 'index'

        get: (args...)->  @tree.get args...
        set: (args...)->  @tree.set args...
    class Manager.Selection
        constructor: (@selection, @name, @tree, config)->
            @tree.set [@name], config
            @cursor = @tree.select [@name]


        update: (selection,selector,data,direction='down')->
            ### Repeatable pattern for creating, updating, and removing data dependent dom elements ###
            [tag,classes...] = selector.split '.'
            selection = selection.selectAll(selector).data data
            if direction in ['right','down']
                selection.enter().append tag
            else if direction in ['left','up']
                selection.enter().insert tag, ':first-child'
            classes.forEach (c)=> selection.classed c, yes
            selection.exit().remove()
            selection

        clear: ()-> @selection.html ''

""").render(**globals())))

# Create Tailored Selections that react to data

There are there selections:  The Table, The Virtual Scoller, and The Bokeh Plot.

In [160]:
IPython.display.Javascript(coffee.compile(jinja2.Template("""    
require ['d3','baobab'], (d3,Baobab)->
    class window.Plot extends Manager.Selection
        config:         
            refs: {{json.dumps(plot)}}
            x: '{{scatter.glyph.x}}'
            y: '{{scatter.glyph.y}}'
            ColumnDataSource: Baobab.monkey ['ColumnDataSource'],['columns'],['index'], (cds,columns,index)->
                updated = {}
                columns.forEach (k)=> updated[k] = cds[k]['data']
                updated

        from_ref = (ref)-> Bokeh.Collections(ref.type).get ref.id                            
        constructor: (selection,name,tree)->  
            super selection, name, tree, @config
            @cds = from_ref @cursor.get 'refs','source'
            @figure = from_ref @cursor.get 'refs','figure'
            @select = @cds.get 'selected'
            @updateDataSource()
            
        updateDataSource: ->
            new_source = @cursor.get 'ColumnDataSource'
            cds = from_ref @cursor.get 'refs','source'
            old_source = cds.get 'data'
            d3.entries new_source
                .forEach (n)=> old_source[n.key] = n.value
            cds.set 'data', old_source
            cds
        updateSelected: ->
            @select['1d']['indices'] = @tree.get 'table','index' 
            @cds.set 'selected', @select
            @cds.trigger 'select'
                        
    class window.Table extends Manager.Selection
        config:
            max_rows: 10
            iloc: 0
            show_id: yes
            index: Baobab.monkey ['.','max_rows'],['.','iloc'],['index'], (rows, iloc, index)-> 
                d3.range iloc, iloc+rows
                    .map (iloc)=> index[iloc]
            columns:
                index: ['index','species']
                exclude: ['sepal_length']
                order: Baobab.monkey ['.','index'],['.','exclude'],['columns'], (index,exclude,columns)->
                    [
                        index
                        columns.filter (c)=> c not in index
                            .filter (c)=> c not in exclude
                    ]
                values: Baobab.monkey ['ColumnDataSource'],['..','index'],['.','order'], (cds, index, order)->
                    v = d3.merge order
                        .map (column)=> 
                            index.map (i)=> cds[column]['data'][i]
                    d3.zip v...
        constructor: (selection,name,tree)->  
            super selection, name, tree, @config
            @tree.set ['table'],@config
            @build()
        build: ->
            table_cursor = @tree.select 'table' 
            table = @update @selection, 'table', [1]
            order = table_cursor.get 'columns','order'
            [left_heading, right_heading] = order
            heading = @update table, 'tr.heading', [order], 'up'
            _t = @
                
            heading.each (columns)->
                _t.update d3.select(@), 'th.index', left_heading,'left'
                    .text (d)-> d
                _t.update d3.select(@), 'th.value', right_heading,'right'
                    .text (d)-> d
            values = @update table, 'tr.values', table_cursor.get('columns','values'), 'down'
            values.each (row_values)->    
                _t.update d3.select(@), 'th.index', left_heading,'left'
                _t.update d3.select(@), 'td.value', right_heading,'right'
                d3.select(@).selectAll 'th.index,td.value'
                    .data row_values
                    .text (d)-> d
    class window.VirtualScroll extends Manager.Selection
        s = (v)->"#{v}px"
        config: 
            speed: 1
            max_rows: Baobab.monkey ['table','max_rows'], (v)-> v
            rowHeight: 20
            height: 800
            width: 600
            scroll_width: 50
            size: Baobab.monkey ['index'], (v)-> v.length
        constructor: (selection,name,tree)->  
            super selection, name, tree, @config
            _this = @
            @cursor = @tree.select ['scroll']
            @cursor.set @config
            @parent = d3.select @selection.property 'parentNode'
            @before = @update @parent, 'div.before.longscroll', [0], 'up'
            @current = @update @parent, 'div.current.longscroll', [0], 'down'
            @after = @update @parent, 'div.after.longscroll', [0], 'down'
            @updateSize()
            console.log @parent
            @parent.on 'scroll.longscroll', ()-> 
                position = Math.floor _this.offset @scrollTop, 'screen-data'
                _this.updateSize()
                _this.scroll @scrollTop,position
            @scroll 0, @tree.get('index',0)
            @build()
        build: ->
            [maxRows,size] = @cursor.project [['max_rows'],['size']]
            @cursor.set ['rowHeight'], d3.mean @selection.selectAll('tr.rows')[0].map (t)-> t.offsetHeight
            @updateSize()
        scroll: (scrollTop,position)->
            @current.call (current)=>
                current.property 'scrollTop', scrollTop
                position = d3.max [0, d3.min [@cursor.get('size') - @cursor.get('max_rows'), position]]
                @before.style("height", s @offset position )
                @after.style("height", s @offset @cursor.get('size')-position)
                
                if Math.abs(position - @tree.get('table','iloc'))>=1
                    @tree.select('table','iloc').set position
                    @render()
        offset: (v,method="data-screen")->
            scale = d3.scale.linear()
                .domain [0,@cursor.get('size')-@cursor.get('max_rows')]
                .range [0,@cursor.get('size')*@cursor.get('size')]
                .clamp yes
            if method in ['data-screen']
                scale v
            else if method in ['screen-data']
                Math.floor scale.invert(v)
        updateSize: ()->
            @selection.style
                position: 'absolute'
                left: s d3.select(@selection.property 'parentNode').property('offsetLeft')
                top: s d3.select(@selection.property 'parentNode').property('offsetTop')
                'margin-top': s 0
            @parent.style 
                'overflow-y': 'auto'              
                height: s @selection.property 'offsetHeight'
                width: s @cursor.get('scroll_width')+@selection.property 'offsetWidth'
                'background-color': 'cyan'  
        render: ()->

""").render(**globals())))

<IPython.core.display.Javascript object>

# All the classes are defined

1. Create a new manager
2. Attach selections
3. Modifies interactions using trees, dispatch, and d3

In [161]:
#{{''.join(bokeh.embed.components(p))}}
IPython.display.display(
IPython.display.HTML(jinja2.Template("""
<div class="vs"></div>
<ul class='buttons'>
<li><a id="shuffle">shuffle</a></li>
</ul>
{{''.join(bokeh.embed.components(p))}}
""").render(**globals())),
IPython.display.Javascript(coffee.compile(jinja2.Template("""    
require ['d3','baobab'], (d3,Baobab)->        
    window.manager = new Manager {{json.dumps(payload)}}
    
    ### Add a Selection to the Manager with ad hoc settings ###
    window.table = manager.attach d3.selectAll('.vs').html(''), Table,'table'
    window.scroller = manager.attach table.selection.select('table'), VirtualScroll,'scroll'  
    window.plot = manager.attach d3.selectAll('.plotdiv'),Plot,'plot'

    #### Shuffle the data ####
    d3.selectAll('#shuffle').on 'click', ()=> 
        manager.DataSource.shuffle()
        plot.updateSelected()
    table.cursor.select 'max_rows'
        .on 'update', ()->  
            table.build()
            scroller.updateSize()

    ### Set up listeners on the tree ###
    manager.DataSource.tree.select('index').on 'update', ()-> 
        table.build()
        
    manager.DataSource.tree.select('data').on 'update', ()-> 
        scroller.updateSize()
    manager.DataSource.tree.select('table','iloc').on 'update', ()-> table.build()
            
    scroller.render = ()-> plot.updateSelected()
    
    #### Fix the table width ###
    table.selection.select('table')
        .style 'width', '900px'
    
    console.log table.selection.node()

""").render(**globals()))))

<IPython.core.display.Javascript object>

# Changing states

Open this notebook locally to change the state of the table by executing the cells below.

In [ ]:
more_rows = """
# Add more rows the table 

        table.cursor.set('max_rows',25)
"""
IPython.display.display(
    IPython.display.Markdown(more_rows),
    IPython.display.Javascript(coffee.compile(more_rows,literate=True)),
)

In [ ]:
more_rows = """
# Sorting
    
1. One Column
    
        manager.DataSource.sort(['sepal_length'])
        alert 'Sorted by sepal_length'

2. Multiple Columns
    
        manager.DataSource.sort(['sepal_length','petal_width','sepal_length'])
        alert "Sorted by ['sepal_length','petal_width','sepal_length']"
        
"""
IPython.display.display(
    IPython.display.Markdown(more_rows),
    IPython.display.Javascript(coffee.compile(more_rows,literate=True)),
)

In [ ]:
more_rows = """
# Add a derived data_source
    cols = ['sepal_length','sepal_width','petal_length','petal_width']
    manager.DataSource.addDerivedDataSource 'mean', cols, (args...)->
            d3.zip args...
                .map (row_value)=> d3.mean row_value
                
# Update the Selections

    table.build()
    scroller.updateSize()
"""
IPython.display.display(
    IPython.display.Markdown(more_rows),
    IPython.display.Javascript(coffee.compile(more_rows,literate=True)),
)
